## Importações

In [2]:
import pandas as pd
import json

## Carregando `applicants.json`

In [3]:
# Carregar o arquivo JSON
with open("dados/applicants.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Lista para armazenar os dados achatados
applicants_list = []

# Iterar sobre cada aplicante no JSON
for applicant_id, details in data.items():
    # Criar dicionário base com o ID do aplicante
    flat_dict = {"id_candidato": applicant_id}
       
    # Iterar sobre cada categoria de informações
    for category, sub_data in details.items():
        if isinstance(sub_data, dict):
            # Se for um dicionário, achatá-lo adicionando o nome da categoria como prefixo
            for key, value in sub_data.items():
                flat_dict[f"{category}_{key}"] = value
        else:
            # Se não for um dicionário, adicionar diretamente
            flat_dict[category] = sub_data
       
    applicants_list.append(flat_dict)

# Criar DataFrame
df = pd.DataFrame(applicants_list)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 58 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   id_candidato                                      42482 non-null  object
 1   infos_basicas_telefone_recado                     42482 non-null  object
 2   infos_basicas_telefone                            42482 non-null  object
 3   infos_basicas_objetivo_profissional               42482 non-null  object
 4   infos_basicas_data_criacao                        42482 non-null  object
 5   infos_basicas_inserido_por                        42482 non-null  object
 6   infos_basicas_email                               42482 non-null  object
 7   infos_basicas_local                               42482 non-null  object
 8   infos_basicas_sabendo_de_nos_por                  42482 non-null  object
 9   infos_basicas_data_atualizac

In [5]:
df.head()

,id_candidato,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,...,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
0,31000,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Verificando porque existem algumas colunas (`cargo_atual_id_ibrati`,`cargo_atual_email_corporativo`,  `cargo_atual_cargo_atual`...) sendo que elas não aparecem por exemplo no primeiro id do json.

Então verifiquei primeiro os que não tem NaN pra poder verificar no json e entender quantos ids tem essa info.

In [6]:
superiores = df[df['cargo_atual_nome_superior_imediato'].notna()]
print(superiores[['id_candidato', 'cargo_atual_nome_superior_imediato']].head())

     id_candidato cargo_atual_nome_superior_imediato
544         31544                                   
814         31814                                   
946         31946                                   
1501         9342                                   
1634         9475                                   


Bom, somente 5 ids possuem alguma informação nesses campos. Se formos verificar no json id por id, podemos ver que a maioria da info está nula ou sem nada. Isso indica que talvez possamos remover essas features.

In [7]:
df[df['id_candidato'] == '31946']

,id_candidato,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,...,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
946,31946,,(11) 98868-1966,Consultora SAP SD,10-12-2021 09:57:59,Yasmin da Rosa,josé_pedro_nunes@gmail.com,"Guarulhos, São Paulo",Anúncio,03-02-2022 18:19:31,...,,,,,,,31-12-1969,31-12-1969,,


## Carregando `prospects.json`

In [8]:
# Carregar o arquivo JSON de prospects
with open("dados/prospects.json", "r", encoding="utf-8") as file:
    data_prospects = json.load(file)

# Lista para armazenar os dados achatados
prospects_list = []

# Iterar sobre cada prospect no JSON
for prospect_id, details in data_prospects.items():
    # Criar dicionário base com o ID do prospect
    flat_dict = {"id": prospect_id}
    
    # Iterar sobre cada categoria de informações
    for category, sub_data in details.items():
        if isinstance(sub_data, dict):
            # Se for um dicionário, achatá-lo adicionando o nome da categoria como prefixo
            for key, value in sub_data.items():
                flat_dict[f"{category}_{key}"] = value
        else:
            # Se não for um dicionário, adicionar diretamente
            flat_dict[category] = sub_data
    
    prospects_list.append(flat_dict)

# Criar DataFrame
df_prospects = pd.DataFrame(prospects_list)


df_prospects

,id,titulo,modalidade,prospects
0,4530,CONSULTOR CONTROL M,,"[{'nome': 'José Vieira', 'codigo': '25632', 's..."
1,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,"[{'nome': 'Sra. Yasmin Fernandes', 'codigo': '..."
2,4532,,,[]
3,4533,2021-2605708-Microfocus Application Life Cycle...,,"[{'nome': 'Arthur Almeida', 'codigo': '26338',..."
4,4534,2021-2605711-Microfocus QTP - UFT Automation T...,,"[{'nome': 'Ana Luiza Vieira', 'codigo': '26361..."
...,...,...,...,...
14217,14218,Java Support Analyst,,"[{'nome': 'Dr. Vitor Hugo Silva', 'codigo': '4..."
14218,14219,,,[]
14219,14220,Consultor Sênior Especialista SAP LES-TRA - 1433,,"[{'nome': 'Ana Cardoso', 'codigo': '16828', 's..."
14220,14221,Consultor Sênior Oracle EPM FCCS - 1434,,"[{'nome': 'Maria Eduarda Cassiano', 'codigo': ..."


In [9]:
# Expandindo a coluna prospects que contém lista de dicionários
def expand_prospects(row):
    if not row['prospects']:  # Se a lista estiver vazia
        return pd.DataFrame([{
            'id': row['id'],
            'titulo': row['titulo'],
            'modalidade': row['modalidade'],
            'candidato_nome': None,
            'candidato_codigo': None,
            'situacao_candidado': None,
            'data_candidatura': None,
            'ultima_atualizacao': None,
            'comentario': None,
            'recrutador': None
        }])
    
    # Criar uma lista de dicionários expandidos
    expanded_rows = []
    for prospect in row['prospects']:
        expanded_rows.append({
            'id': row['id'],
            'titulo': row['titulo'],
            'modalidade': row['modalidade'],
            'candidato_nome': prospect.get('nome'),
            'candidato_codigo': prospect.get('codigo'),
            'situacao_candidado': prospect.get('situacao_candidado'),
            'data_candidatura': prospect.get('data_candidatura'),
            'ultima_atualizacao': prospect.get('ultima_atualizacao'),
            'comentario': prospect.get('comentario'),
            'recrutador': prospect.get('recrutador')
        })
    
    return pd.DataFrame(expanded_rows)

# Aplicando a expansão
df_prospects_expanded = pd.concat(df_prospects.apply(expand_prospects, axis=1).tolist(), ignore_index=True)

# Exibindo as primeiras linhas do DataFrame expandido
print("\nPrimeiras linhas do DataFrame de Prospects expandido:")
print(df_prospects_expanded.head())

# Exibindo informações sobre o DataFrame expandido
print("\nInformações do DataFrame de Prospects expandido:")
print(df_prospects_expanded.info())


Primeiras linhas do DataFrame de Prospects expandido:
     id                                             titulo modalidade  \
0  4530                                CONSULTOR CONTROL M              
1  4530                                CONSULTOR CONTROL M              
2  4531  2021-2607395-PeopleSoft Application Engine-Dom...              
3  4531  2021-2607395-PeopleSoft Application Engine-Dom...              
4  4532                                                                 

             candidato_nome candidato_codigo           situacao_candidado  \
0               José Vieira            25632  Encaminhado ao Requisitante   
1  Srta. Isabela Cavalcante            25529  Encaminhado ao Requisitante   
2     Sra. Yasmin Fernandes            25364     Contratado pela Decision   
3            Alexia Barbosa            25360  Encaminhado ao Requisitante   
4                      None             None                         None   

  data_candidatura ultima_atualizacao  \
0 

In [10]:
df_prospects_expanded

,id,titulo,modalidade,candidato_nome,candidato_codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano
4,4532,,,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
56697,14219,,,None,None,None,None,None,None,None
56698,14220,Consultor Sênior Especialista SAP LES-TRA - 1433,,Ana Cardoso,16828,Desistiu,26-02-2025,28-02-2025,Recebeu a confirmação de outro processo seleti...,Elisa Nunes
56699,14220,Consultor Sênior Especialista SAP LES-TRA - 1433,,Pedro Lucas das Neves,15042,Encaminhado ao Requisitante,28-02-2025,28-02-2025,,Elisa Nunes
56700,14221,Consultor Sênior Oracle EPM FCCS - 1434,,Maria Eduarda Cassiano,49190,Prospect,26-02-2025,26-02-2025,,Luna Correia


## Carregando `vagas.json`

In [11]:
# Carregar o arquivo JSON de vagas
with open("dados/vagas.json", "r", encoding="utf-8") as file:
    data_vagas = json.load(file)

# Lista para armazenar os dados achatados
vagas_list = []

# Iterar sobre cada vaga no JSON
for vaga_id, details in data_vagas.items():
    # Criar dicionário base com o ID da vaga
    flat_dict = {"id_vaga": vaga_id}
    
    # Iterar sobre cada categoria de informações
    for category, sub_data in details.items():
        if isinstance(sub_data, dict):
            # Se for um dicionário, achatá-lo adicionando o nome da categoria como prefixo
            for key, value in sub_data.items():
                flat_dict[f"{category}_{key}"] = value
        else:
            # Se não for um dicionário, adicionar diretamente
            flat_dict[category] = sub_data
    
    vagas_list.append(flat_dict)

# Criar DataFrame
df_vagas = pd.DataFrame(vagas_list)
df_vagas

,id_vaga,informacoes_basicas_data_requicisao,informacoes_basicas_limite_esperado_para_contratacao,informacoes_basicas_titulo_vaga,informacoes_basicas_vaga_sap,informacoes_basicas_cliente,informacoes_basicas_solicitante_cliente,informacoes_basicas_empresa_divisao,informacoes_basicas_requisitante,informacoes_basicas_analista_responsavel,...,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,informacoes_basicas_data_inicial,informacoes_basicas_data_final,perfil_vaga_habilidades_comportamentais_necessarias,informacoes_basicas_nome_substituto
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,...,• Início: Imediato • Fim: Jan/22,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,- p/ mês (168h),fechado,,18-05-2021,17-01-2022,NaN,NaN
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14076,12368,28-02-2024,31-03-2024,PMP 5203658 - Dynamics,Não,Miller-Curry,Guilherme Campos,Decision Campinas,Sra. Juliana Oliveira,Sra. Joana Sousa,...,,,,"123,00 -",APPLICATION DEVELOPER-MICROSOFT ANALYTICS - ba...,,NaN,NaN,NaN,NaN
14077,12367,28-02-2024,29-02-2024,MM com IM - FSR 3910,Não,Bishop-Reed,Lívia Vasconcelos,Decision São Paulo,Manuella Jesus,Srta. Bella Ferreira,...,FSR: 3910,Não,,168 -,Fechado,,NaN,NaN,NaN,NaN
14078,12366,27-02-2024,29-02-2024,964 - Assistente fiscal ou financeiro,Não,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Maria Clara Pires,...,"Modelo de trabalho: Hibrido, 3 vezes por seman...",Não,,168 -,Fechado,,NaN,NaN,NaN,NaN
14079,12365,27-02-2024,29-02-2024,966 - Analista pleno ou sênior de área fiscal,Não,"Glover, Warren and Norris",Lorenzo Cunha,Decision São Paulo,Maria Lopes,Melina Montenegro,...,"Modelo de trabalho: Hibrido, 3 vezes por seman...",Não,,168 -,Fechado,,NaN,NaN,NaN,NaN


## Juntando as tabelas

`id_vaga` é o mesmo do `id` da tabela de prospects. E na tabela prospects o `codigo_candidato` é ligado ao `id_candidato` da tabela de aplicantes.


In [12]:
df_prospects_expanded = df_prospects_expanded.rename(columns={'id': 'id_vaga'})

In [13]:
# Convertendo o id_vaga para string em ambos os DataFrames para garantir compatibilidade
df_vagas['id_vaga'] = df_vagas['id_vaga'].astype(str)
df_prospects_expanded['id_vaga'] = df_prospects_expanded['id_vaga'].astype(str)

In [14]:
# Unindo vagas com prospects
df_vagas_prospects = pd.merge(
    df_prospects_expanded,
    df_vagas,
    on='id_vaga',
    how='left'
)

In [15]:
df_vagas_prospects.head()

,id_vaga,titulo,modalidade,candidato_nome,candidato_codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,...,perfil_vaga_demais_observacoes,perfil_vaga_viagens_requeridas,perfil_vaga_equipamentos_necessarios,beneficios_valor_venda,beneficios_valor_compra_1,beneficios_valor_compra_2,informacoes_basicas_data_inicial,informacoes_basicas_data_final,perfil_vaga_habilidades_comportamentais_necessarias,informacoes_basicas_nome_substituto
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,...,Contratação PJ Projeto pontual de 2 a 3 meses ...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,...,Contratação PJ Projeto pontual de 2 a 3 meses ...,,Nenhum -,-,R$,,NaN,NaN,NaN,NaN
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,...,"Remoto DEPOIS PRESENCIAL, TEMPO INDETERMINADO",,Notebook padrão -,-,hora,,NaN,NaN,NaN,NaN
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano,...,"Remoto DEPOIS PRESENCIAL, TEMPO INDETERMINADO",,Notebook padrão -,-,hora,,NaN,NaN,NaN,NaN
4,4532,,,None,None,None,None,None,None,None,...,APENAS PJ E CLT FULL!!!,,NaN,-,Valor Aberto,,NaN,NaN,Tempo de projeto = 1 ano\nRequested start date...,NaN


In [16]:
df_final = pd.merge(
    df_vagas_prospects,
    df,
    left_on='candidato_codigo',
    right_on='id_candidato',
    how='left'
)

df_final.head()

,id_vaga,titulo,modalidade,candidato_nome,candidato_codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,...,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4532,,,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56702 entries, 0 to 56701
Columns: 112 entries, id_vaga to cargo_atual_email_superior_imediato
dtypes: object(112)
memory usage: 48.5+ MB


In [18]:
df_final.to_csv('merged_data.csv', index=False)